# Imports

In [1]:
import numpy as np
import os
import PIL
import tempfile
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import time

from django.shortcuts import render
from keras.models import load_model

2023-01-19 18:14:54.957252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 18:14:55.119434: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-19 18:14:55.119456: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-19 18:14:56.012532: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Load model

In [2]:
os.chdir("/home/sam/Documents/cloud-computing-project/Project/backend/") # path to the folder of the model
model = load_model("Model_Suspect_Detection.h5")

2023-01-19 18:15:01.328964: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-19 18:15:01.328990: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-19 18:15:01.329014: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sam-laptop): /proc/driver/nvidia/version does not exist
2023-01-19 18:15:01.329240: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Prune model

In [9]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1  # 10% of training set will be used for validation set.

# Count number of images used for training
dirs_count = 0
files_count = 0
for path, dirs, files in os.walk("/home/sam/Documents/cloud-computing-project/Project/backend/databases/Big_Suspect_Database/train"):
    dirs_count += len(dirs)
    files_count += len(files)
print(files_count)

num_images = files_count * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                             final_sparsity=0.80,
                                                             begin_step=0,
                                                             end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# 'prune_low_magnitude' requires a recompile.
model_for_pruning.compile(optimizer='adam',
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])
model_for_pruning.summary()

5428
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 prune_low_magnitude_zero_paddi  (None, 230, 230, 3)  1          ['input_1[0][0]']                
 ng2d (PruneLowMagnitude)                                                                         
                                                                                                  
 prune_low_magnitude_conv1/conv  (None, 112, 112, 64  18818      ['prune_low_magnitude_zero_paddin
  (PruneLowMagnitude)           )                                g2d[0][0]']             

 k3_1_relu (PruneLowMagnitude)                                   3_1_bn[0][0]']                   
                                                                                                  
 prune_low_magnitude_conv2_bloc  (None, 56, 56, 32)  73730       ['prune_low_magnitude_conv2_block
 k3_2_conv (PruneLowMagnitude)                                   3_1_relu[0][0]']                 
                                                                                                  
 prune_low_magnitude_conv2_bloc  (None, 56, 56, 160)  1          ['prune_low_magnitude_conv2_block
 k3_concat (PruneLowMagnitude)                                   2_concat[0][0]',                 
                                                                  'prune_low_magnitude_conv2_block
                                                                 3_2_conv[0][0]']                 
                                                                                                  
 prune_low

  (PruneLowMagnitude)                                            [0]']                            
                                                                                                  
 prune_low_magnitude_pool2_conv  (None, 56, 56, 128)  65538      ['prune_low_magnitude_pool2_relu[
  (PruneLowMagnitude)                                            0][0]']                          
                                                                                                  
 prune_low_magnitude_pool2_pool  (None, 28, 28, 128)  1          ['prune_low_magnitude_pool2_conv[
  (PruneLowMagnitude)                                            0][0]']                          
                                                                                                  
 prune_low_magnitude_conv3_bloc  (None, 28, 28, 128)  513        ['prune_low_magnitude_pool2_pool[
 k1_0_bn (PruneLowMagnitude)                                     0][0]']                          
          

 prune_low_magnitude_conv3_bloc  (None, 28, 28, 128)  57346      ['prune_low_magnitude_conv3_block
 k4_1_conv (PruneLowMagnitude)                                   4_0_relu[0][0]']                 
                                                                                                  
 prune_low_magnitude_conv3_bloc  (None, 28, 28, 128)  513        ['prune_low_magnitude_conv3_block
 k4_1_bn (PruneLowMagnitude)                                     4_1_conv[0][0]']                 
                                                                                                  
 prune_low_magnitude_conv3_bloc  (None, 28, 28, 128)  1          ['prune_low_magnitude_conv3_block
 k4_1_relu (PruneLowMagnitude)                                   4_1_bn[0][0]']                   
                                                                                                  
 prune_low_magnitude_conv3_bloc  (None, 28, 28, 32)  73730       ['prune_low_magnitude_conv3_block
 k4_2_conv

                                                                  'prune_low_magnitude_conv3_block
                                                                 7_2_conv[0][0]']                 
                                                                                                  
 prune_low_magnitude_conv3_bloc  (None, 28, 28, 352)  1409       ['prune_low_magnitude_conv3_block
 k8_0_bn (PruneLowMagnitude)                                     7_concat[0][0]']                 
                                                                                                  
 prune_low_magnitude_conv3_bloc  (None, 28, 28, 352)  1          ['prune_low_magnitude_conv3_block
 k8_0_relu (PruneLowMagnitude)                                   8_0_bn[0][0]']                   
                                                                                                  
 prune_low_magnitude_conv3_bloc  (None, 28, 28, 128)  90114      ['prune_low_magnitude_conv3_block
 k8_1_conv

...

 prune_low_magnitude_conv5_bloc  (None, 7, 7, 1472)  5889        ['prune_low_magnitude_conv5_block
 k19_0_bn (PruneLowMagnitude)                                    18_concat[0][0]']                
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 1472)  1           ['prune_low_magnitude_conv5_block
 k19_0_relu (PruneLowMagnitude)                                  19_0_bn[0][0]']                  
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 128)   376834      ['prune_low_magnitude_conv5_block
 k19_1_conv (PruneLowMagnitude)                                  19_0_relu[0][0]']                
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 128)   513         ['prune_low_magnitude_conv5_block
 k19_1_bn 

                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 32)    73730       ['prune_low_magnitude_conv5_block
 k22_2_conv (PruneLowMagnitude)                                  22_1_relu[0][0]']                
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 1600)  1           ['prune_low_magnitude_conv5_block
 k22_concat (PruneLowMagnitude)                                  21_concat[0][0]',                
                                                                  'prune_low_magnitude_conv5_block
                                                                 22_2_conv[0][0]']                
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 1600)  6401        ['prune_low_magnitude_conv5_block
 k23_0_bn 

                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 128)   434178      ['prune_low_magnitude_conv5_block
 k26_1_conv (PruneLowMagnitude)                                  26_0_relu[0][0]']                
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 128)   513         ['prune_low_magnitude_conv5_block
 k26_1_bn (PruneLowMagnitude)                                    26_1_conv[0][0]']                
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 128)   1           ['prune_low_magnitude_conv5_block
 k26_1_relu (PruneLowMagnitude)                                  26_1_bn[0][0]']                  
                                                                                                  
 prune_low

 k29_concat (PruneLowMagnitude)                                  28_concat[0][0]',                
                                                                  'prune_low_magnitude_conv5_block
                                                                 29_2_conv[0][0]']                
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 1824)  7297        ['prune_low_magnitude_conv5_block
 k30_0_bn (PruneLowMagnitude)                                    29_concat[0][0]']                
                                                                                                  
 prune_low_magnitude_conv5_bloc  (None, 7, 7, 1824)  1           ['prune_low_magnitude_conv5_block
 k30_0_relu (PruneLowMagnitude)                                  30_0_bn[0][0]']                  
                                                                                                  
 prune_low

 prune_low_magnitude_dense (Pru  (None, 1024)        3933186     ['prune_low_magnitude_global_aver
 neLowMagnitude)                                                 age_pooling2d[0][0]']            
                                                                                                  
 prune_low_magnitude_dense_1 (P  (None, 9)           18443       ['prune_low_magnitude_dense[0][0]
 runeLowMagnitude)                                               ']                               
                                                                                                  
Total params: 40,138,392
Trainable params: 20,069,257
Non-trainable params: 20,069,135
__________________________________________________________________________________________________


In [ ]:
log_dir = tempfile.mkdtemp()
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir),
]

batch_size = 128  # @param [1,2,4,8,16,32,64,128] {type:"raw"}
epochs = 2  # @param [2, 5, 10,20,50,100,200] {type:"raw"}
dataset_name = '/home/sam/Documents/cloud-computing-project/Project/backend/databases/Big_Suspect_Database'
train_dataset = os.path.join(dataset_name, 'train/')
test_dataset = os.path.join(dataset_name, 'test/')
input_dim = 255  # @param [224,299] {type:"raw"}

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(input_dim, input_dim),
    batch_size=batch_size,
    label_mode='categorical',
)

train_ds = train_ds.unbatch()
train_images = list(train_ds.map(lambda x, y: x))
train_labels = list(train_ds.map(lambda x, y: y))

model_for_pruning.fit(train_images, train_labels,
                     batch_size=batch_size, epochs=epochs,
                     validation_split=validation_split,
                     callbacks=callbacks)

Found 5427 files belonging to 9 classes.
Using 4342 files for training.


2023-01-19 18:50:36.792301: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-01-19 18:50:37.997576: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-01-19 18:50:39.916815: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-01-19 18:50:41.935371: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-01-19 18:50:42.474623: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-01-19 18:50:44.739072: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-01-19 18:50:49.671975: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-01-19 18:50:50.489305: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-01-19 18:50:51.318673: W tensorflow/core/lib/png/png_io.cc:88] PNG 